In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
#from utils import *
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.utils import from_networkx
from statsmodels.stats.outliers_influence import variance_inflation_factor
import random
import itertools
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
tex_fonts = {
    #source: https://jwalton.info/Embed-Publication-Matplotlib-Latex/
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    #"axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "axes.titlesize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    'text.latex.preamble': r"\usepackage{amsmath}"
}
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Read the CSV file
csv_file = 'LondonLSOAs.csv'
df = pd.read_csv(csv_file)

# Convert the 'geometry' column to Shapely geometry objects
df['geometry'] = df['geometry'].apply(wkt.loads)

# Create the GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Display the GeoDataFrame to ensure it's correct
print(gdf.head())

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkt
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib_scalebar.scalebar as sb

# Read CSV file and create GeoDataFrame
csv_file = 'LondonLSOAs.csv'
df = pd.read_csv(csv_file)

# Create geometry objects (assuming the geometry column contains polygon data in WKT format)
gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']))

# Set the coordinate system (assuming it is WGS84 coordinate system)
gdf.set_crs(epsg=27700, inplace=True)

# Calculate the geometric center point of each LSOA
gdf['geo_centroid'] = gdf.centroid

# Calculate neighbor relationships
neighbors = gpd.sjoin(gdf, gdf, how='left', predicate='intersects')

# Manually process indexes
neighbors = neighbors[neighbors.index != neighbors.index_right]

# Build adjacency matrix
G = nx.Graph()

# Add nodes
for idx, row in gdf.iterrows():
G.add_node(idx, pos=(row.geo_centroid.x, row.geo_centroid.y))

# Add edges based on neighbor relationships
for idx, row in neighbors.iterrows():
G.add_edge(row.name, row['index_right'], weight=1)

# Visualize neighbor relationships
fig, ax = plt.subplots(figsize=(10, 10))

# Draw geographic coordinate borders and set transparency
gdf.boundary.plot(ax=ax, linewidth=0.6, edgecolor='black')

# Draw nodes and edges
pos = nx.get_node_attributes(G, 'pos')
nx.draw(G, pos, node_size=3, node_color='blue', edge_color='grey', alpha=0.5, ax=ax)

# Add a scale
scalebar = sb.ScaleBar(1, location='lower right')
ax.add_artist(scalebar)

# Create a custom legend
legend_elements = [
Line2D([0], [0], marker='o', color='w', label='LSOA Nodes',
markerfacecolor='blue', markersize=10),
Line2D([0], [0], color='grey', lw=2, label='Connect Edges'),
Patch(facecolor='none', edgecolor='black', label='LSOA Boundary')
]

# Add legend to the figure
ax.legend(handles=legend_elements, loc='upper right')

plt.title("Single-layer Adjacency Network")

# # Make sure the target folder exists
# output_folder = 'Paper output'
# os.makedirs(output_folder, exist_ok=True)

# # Save the image to the target folder
# output_path = os.path.join(output_folder, 'Single-layer Adjacency Network.png')
# plt.savefig(output_path)

plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx
import matplotlib.pyplot as plt

# Read CSV file
csv_file = 'LondonLSOAs.csv'
df = pd.read_csv(csv_file)

# Create GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set coordinate system (assuming WGS84)
gdf.set_crs(epsg=27700, inplace=True)

# Calculate geometric center point
gdf['geo_centroid'] = gdf.centroid
coords = np.array(list(zip(gdf.geo_centroid.x, gdf.geo_centroid.y)))

# Calculate the distance between centroids
dist_matrix = distance_matrix(coords, coords)

# Create a network
G_knn = nx.Graph()

# Add nodes
for idx, row in gdf.iterrows():
G_knn.add_node(idx, pos=(row.geo_centroid.x, row.geo_centroid.y))

# KNN settings
k = 5 
# K = 3
# Build a KNN graph based on the distance matrix
for i in range(len(gdf)):
# Get the distance from the current node to all other nodes
distances = dist_matrix[i]

# Find the indices of the K nearest neighbors
knn_indices = np.argsort(distances)[1:k+1] # Sort and take the first K (excluding itself)

# Add edges
for neighbor in knn_indices:
G_knn.add_edge(i, neighbor, weight=1 / distances[neighbor]) # Weight is the inverse of distance

# Visualize KNN graph
pos_knn = nx.get_node_attributes(G_knn, 'pos')
plt.figure(figsize=(10, 10))
nx.draw(G_knn, pos_knn, node_size=10, node_color='blue', edge_color='grey', alpha=0.5)
plt.title("KNN Graph with K=5")
plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx
import matplotlib.pyplot as plt

# Read CSV file
csv_file = 'LondonLSOAs.csv'
df = pd.read_csv(csv_file)

# Create GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set coordinate system (assuming WGS84)
gdf.set_crs(epsg=4326, inplace=True)

# Calculate geometric center point
gdf['geo_centroid'] = gdf.centroid
coords = np.array(list(zip(gdf.geo_centroid.x, gdf.geo_centroid.y)))

# Calculate neighbor relationships
neighbors = gpd.sjoin(gdf, gdf, how='left', predicate='intersects')
neighbors = neighbors[neighbors.index != neighbors.index_right]

# Calculate the distance between center points
dist_matrix = distance_matrix(coords, coords)

# Create a neighbor relationship dictionary
first_level_neighbors = {i: [] for i in range(len(gdf))}
second_level_neighbors = {i: set() for i in range(len(gdf))}

for idx, row in neighbors.iterrows():
first_level_neighbors[row.name].append(row['index_right'])

# Find the second-level neighbors
for i in first_level_neighbors:
for first_neighbor in first_level_neighbors[i]:
second_level_neighbors[i].update(first_level_neighbors[first_neighbor])
second_level_neighbors[i].discard(i)
second_level_neighbors[i].difference_update(first_level_neighbors[i])

# Build a network and set weights based on the inverse of the distance
G = nx.Graph()

# Add nodes
for idx, row in gdf.iterrows():
G.add_node(idx, pos=(row.geo_centroid.x, row.geo_centroid.y))

# Add the first-level neighbor edge with high weight
for idx, neighbors in first_level_neighbors.items():
for neighbor in neighbors:
if dist_matrix[idx][neighbor] > 0: # Ensure distance is valid
G.add_edge(idx, neighbor, weight=0.7)

# Add edge of second level neighbors, with low weight
for idx, neighbors in second_level_neighbors.items():
for neighbor in neighbors:
if dist_matrix[idx][neighbor] > 0: # Ensure distance is valid
G.add_edge(idx, neighbor, weight=0.3) # Low weight

# Visualize neighbor relationships
pos = nx.get_node_attributes(G, 'pos')
plt.figure(figsize=(10, 10))
nx.draw(G, pos, node_size=10, node_color='blue', edge_color='grey', alpha=0.5) plt.title("Adjacency Network with Weighted Edges") plt.show()